# 构造微调训练数据集

借助 ChatGPT 和 GPT API 我们可以实现自动化批量构造训练数据集。

下面我们以中国古典哲学数据集为例，展示了自动构造训练集的主要流程：

- 使用 LangChain 构造训练数据样例
    - 基于 ChatGPT 设计 `System Role` 提示词
    - 使用 `OpenAI GPT-4o-mini` 生成基础数据
    - 解析 OpenAI GPT 生成的训练数据
    - 持久化存储`dataset.csv`训练数据集文件
    - 使用 ChatGPT 实现训练数据多样化
- 自动化批量生成训练数据集
    - 整理收集原始数据`raw_data.txt`
    - 自动解析原始数据样例 `raw_data_content[]`
    - 设计 `gen_data` 训练数据生成器函数
    - 设计训练数据生成流水线

使用langchain 对接阿里的百炼平台api，使用新的数据，生成新的数据集

## 使用 OpenAI SDk 构造训练数据

In [23]:
from langchain_openai import ChatOpenAI
client = ChatOpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-1c7dd91af88b47ec94b72a887c7d2581",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_proxy=None   # 显式设置代理为None
)

---

## 使用 LangChain 构造训练数据

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

client = ChatOpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    model="qwen-plus",
    temperature=1,
    max_tokens=4095,
    api_key="sk-1c7dd91af88b47ec94b72a887c7d2581",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_proxy=None   # 显式设置代理为None
)                  

In [25]:
system_content = """
你是中国古典哲学大师，尤其擅长周易的哲学解读。

接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

示例输入：

师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

期待结果：

content:"师卦"
summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
"""


In [26]:
# 原始数据
raw_content = "谦卦：通泰。筮遇此卦，君子将有所成就。《象辞》说：本卦外卦为坤为地，内卦为艮为山，地中有山，内高外卑，居高不傲，这是谦卦的卦象。君子观此卦象，以谦让为怀，裁取多余昀，增益缺乏的，衡量财物的多寡而公平施予。"

In [27]:
messages = [
    SystemMessage(
        content=system_content
    ),
    HumanMessage(
        content=raw_content
    ),
]

In [28]:
ai_message = client.invoke(messages)

### 解析 OpenAI GPT 生成的训练数据

In [29]:
ai_message.content

'content:"谦卦"\nsummary:"在《周易》中，谦卦象征着通达与安宁。此卦的结构是外卦为坤（地），内卦为艮（山），整体呈现出“地中有山”的意象。这种形象描绘出山藏于地之中，象征着内里崇高而外表谦卑，正是谦德之象。\n\n《象辞》指出，君子观此卦象，应当以谦逊为本，像山一样稳重而内敛。同时，君子应当根据财物的多少，适当地裁减有余、补足不足，做到公平施与，体现出一种社会正义与资源平衡的理念。\n\n谦卦的核心哲学在于强调低调与节制，不炫耀、不争功，居高位而不自傲，处困境而不自卑。这种德性不仅是个人修养的重要体现，也是治国理政的根本原则之一。因此，若君子得此卦象，预示着将有所成就，因其以谦德行于世，必能赢得人心，成就大业。"'

In [31]:
text = ai_message.content

# 分割字符串来找到content和summary的位置
content_start = text.find('content:"') + len('content:"')
content_end = text.find('"\nsummary:')
summary_start = text.find('summary:"') + len('summary:"')
summary_end = text.rfind('"')

# 提取并存储content和summary
content = text[content_start:content_end].strip()
summary = text[summary_start:summary_end].strip()

print("Content:", content)
print("Summary:", summary)


Content: 谦卦
Summary: 在《周易》中，谦卦象征着通达与安宁。此卦的结构是外卦为坤（地），内卦为艮（山），整体呈现出“地中有山”的意象。这种形象描绘出山藏于地之中，象征着内里崇高而外表谦卑，正是谦德之象。

《象辞》指出，君子观此卦象，应当以谦逊为本，像山一样稳重而内敛。同时，君子应当根据财物的多少，适当地裁减有余、补足不足，做到公平施与，体现出一种社会正义与资源平衡的理念。

谦卦的核心哲学在于强调低调与节制，不炫耀、不争功，居高位而不自傲，处困境而不自卑。这种德性不仅是个人修养的重要体现，也是治国理政的根本原则之一。因此，若君子得此卦象，预示着将有所成就，因其以谦德行于世，必能赢得人心，成就大业。


### 持久化存储训练数据集文件

In [35]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

# 新建CSV文件并写入数据
with open('new_test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 写入标题行
    writer.writerow(['content', 'summary'])
    # 写入数据行
    writer.writerow([content, summary])


### 数据增强：构造多样化的提问方式

In [36]:
def generate_question_summary_pairs(content, summary):
    """
    生成20对提问和总结的配对。

    :param content: 内容（例如：“蒙卦”）。
    :param summary: 内容的总结。
    :return: 包含20对提问和总结的列表。
    """
    # 20种提问模板
    question_templates = [
        "{}代表什么？",
        "周易中的{}含义是什么？",
        "请解释一下{}。",
        "{}在周易中是什么象征？",
        "周易{}的深层含义是什么？",
        "{}和教育启蒙有什么联系？",
        "周易的{}讲述了什么？",
        "{}是怎样的一个卦象？",
        "{}在周易中怎样表达教育的概念？",
        "{}的基本意义是什么？",
        "周易中{}的解释是什么？",
        "{}在周易中代表了哪些方面？",
        "{}涉及哪些哲学思想？",
        "周易中{}的象征意义是什么？",
        "{}的主要讲述内容是什么？",
        "周易{}的核心思想是什么？",
        "{}和启蒙教育之间有何联系？",
        "在周易中，{}象征着什么？",
        "请描述{}的含义。",
        "{}在周易哲学中扮演什么角色？"
    ]

    # 使用content填充提问模板
    questions = [template.format(content) for template in question_templates]

    # 创建提问和总结的配对
    question_summary_pairs = [(question, summary) for question in questions]

    return question_summary_pairs

In [37]:
import csv

# 如果没有GPT API，可以使用预定义的变量
# content = "蒙卦"
# summary = "在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
pairs = generate_question_summary_pairs(content, summary)

# 将结果写入CSV文件
with open('new_test_dataset.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['content', 'summary'])
    for pair in pairs:
        writer.writerow(pair)


## 自动化批量生成训练数据流水线

原始数据来源：https://www.zhouyi.cc/zhouyi/yijing64/4103.html

In [38]:
# 初始化一个空列表用于存储原始内容数据
raw_content_data = []

# 读取文件并分割数据样例
with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    # 使用连续的换行符('\n\n')作为分隔符来分割文本
    data_samples = content.split('\n\n')

    # 遍历分割后的数据样例并添加到列表中
    for sample in data_samples:
        # 移除每个样例中的额外空白字符（如果有的话）
        cleaned_sample = sample.strip()
        # 仅添加非空样例
        if cleaned_sample:
            raw_content_data.append(cleaned_sample)

In [39]:
# 输出结果以验证
for i, sample in enumerate(raw_content_data[:5]):  # 打印前5个样例以检查
    print(f"样例 {i+1}:")
    print(sample)
    print("------")


样例 1:
谦卦原文
谦。亨，君子有终。
象曰：地中有山，谦。君子以裒多益寡，称物平施。
白话文解释
谦卦：通泰。筮遇此卦，君子将有所成就。
《象辞》说：本卦外卦为坤为地，内卦为艮为山，地中有山，内高外卑，居高不傲，这是谦卦的卦象。君子观此卦象，以谦让为怀，裁取多余昀，增益缺乏的，衡量财物的多寡而公平施予。
《断易天机》解
谦卦坤上艮下，为兑宫五世卦。谦为谦逊，内卦艮和外卦坤都是谦虚之象，所以为吉利之卦。
北宋易学家邵雍解
谦和忍让，尊人自卑；利用谦虚，万事可达。
得此卦者，吉利平安，步步高升。谦虚忍让者前途大好，骄横者必招败。谦受益，满招损。
台湾国学大儒傅佩荣解
时运：眼前平顺，步步高升。
财运：物价合理，利益保障。
家宅：近山之居，合家平安。
身体：内郁之症，放宽心胸。
传统解卦
这个卦是异卦（下艮上坤）相叠，艮为山，坤为地。地面有山，地卑（低）而山高，是为内高外低，比喻功高不自居，名高不自誉，位高不自傲。这就是谦。
大象：山应在地面上，而今在地下，代表谦逊之德。
运势：吉利平安，前途无限，但必须去除骄横之气。
事业：尚未被人器重，但因品德高尚，终会为人发现。自己不必有意表现，尤其不可放弃谦虚的美德，埋头苦干，一定会得到他人的帮助，在事业上大有作为。
经商：较为顺利，但不会获暴利，须逐步积累，故应极其小心谨慎，诚心与他人合作，遵守商业道德，通过自己的勤劳努力，取得商业的发展。
求名：天资聪明，但因性格不善于表现自己，起初往往被埋没，自己只要不灰心，继续努力钻研业务，保持谦逊好学精神，更大的成功正在未来。
婚恋：美满的婚姻应由双方共同去努力争取。否则，恋爱不成功，家庭不能和睦。
决策：具有内高外低的谦虚美德，善于团结他人，也能得到别人的帮助，事业很顺利，遇到困难，很快可以化险为夷，有惊无险。由于能够认真吸取别人的长处，进步很快，会畅通无阻，但需要特别注意学业、德业和修养。
------
样例 2:
恒卦原文
恒。亨，无咎，利贞。利有攸往。
象曰：雷风，恒。君子以立不易方。
白话文解释
恒卦：通达，没有过失，吉利的卜问。有所往则有利。
《象辞》说：本卦上卦为震，震为雷，下卦为巽，巽为风，风雷荡涤，宇宙常新，这是恒卦的卦象。君子观此卦象，从而立于正道，坚守不易。
《断易天机》解
恒卦震上巽下，为震宫三世卦。恒卦的意思就是要坚持、恒定、有恒心，这样才能获得亨通。
北宋

### 将以上的所有模块，整合到一起，自动化生成数据

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

# 初始化LangChain的GPT-4o-mini调用

client = ChatOpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    model="qwen-plus",
    temperature=1,
    max_tokens=4095,
    api_key="sk-1c7dd91af88b47ec94b72a887c7d2581",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_proxy=None   # 显式设置代理为None
)  

def gen_data(raw_content):
    """
    使用LangChain GPT-4o-mini调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-4o-mini模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = client.invoke(messages)

    return ai_message.content

In [41]:
# 示例调用（使用 raw_data.txt 中解析的数据样例）
generated_content = gen_data(raw_content_data[0])
print(generated_content)

content:"谦卦"
summary:"谦卦是《周易》六十四卦之一，由异卦相叠而成，下卦为艮（山），上卦为坤（地），形成“地中有山”的卦象。这一卦象象征着谦逊之美德——内里高峻如山，外在谦卑如地，体现了一种低调自持、内敛修养的人格风范。

卦辞曰：“谦，亨，君子有终。”意指谦虚之道可以通达顺利，君子若能始终如一地保持谦逊，终将有所成就。《象辞》进一步阐释此卦象为：“地中有山，谦。君子以裒多益寡，称物平施。”这表明君子应当效法自然的平衡之道，损有余而补不足，公平施予，以谦让为本，协调人际关系与社会资源。

从卦象结构来看，谦卦是唯一全吉的卦，六爻皆吉，象征无论在何种处境中，只要持守谦德，皆可逢凶化吉、转危为安。北宋易学家邵雍称其为“谦和忍让，尊人自卑；利用谦虚，万事可达”。得此卦者，若能守谦虚之道，前途将一片光明，而骄傲自满者则易招致失败。

在运势上，谦卦象征吉利平安、步步高升，宜守柔、宜退让、宜积蓄力量，不宜冒进张扬。在事业方面，虽然暂时可能未被重视，但凭借高尚的德行与踏实的努力，终将获得他人的认可与助力，成就一番事业。在经商方面，则预示平稳发展、逐步积累，需谨慎经营、诚心合作，方可长久获利。在求名方面，虽可能一时不被赏识，但若坚持努力与谦逊，终将脱颖而出。在婚恋方面，则提示双方需共同经营、相互谦让，方能收获美满的家庭生活。

总体而言，谦卦所传达的核心哲学是：谦逊是通达之道，是成功之基，是君子修身齐家治国平天下的重要德行。它不仅是一种处世智慧，更是一种内在修养的体现，具有极高的道德价值与实践指导意义。"


In [42]:
def dataset_parser(ai_message_content):
    """
    解析由gen_data函数生成的ai_message.content，提取content和summary。

    :param ai_message_content: gen_data函数返回的文本。
    :return: 提取的content和summary。
    """
    # 分割字符串来找到content和summary的位置
    content_start = ai_message_content.find('content:"') + len('content:"')
    content_end = ai_message_content.find('"\nsummary:')
    summary_start = ai_message_content.find('summary:"') + len('summary:"')
    summary_end = ai_message_content.rfind('"')

    # 提取并存储content和summary
    content = ai_message_content[content_start:content_end].strip()
    summary = ai_message_content[summary_start:summary_end].strip()

    return content, summary


In [43]:
# 示例调用（使用假设的gen_data函数返回的文本）
content, summary = dataset_parser(generated_content)
print("Content:", content)
print("Summary:", summary)

Content: 谦卦
Summary: 谦卦是《周易》六十四卦之一，由异卦相叠而成，下卦为艮（山），上卦为坤（地），形成“地中有山”的卦象。这一卦象象征着谦逊之美德——内里高峻如山，外在谦卑如地，体现了一种低调自持、内敛修养的人格风范。

卦辞曰：“谦，亨，君子有终。”意指谦虚之道可以通达顺利，君子若能始终如一地保持谦逊，终将有所成就。《象辞》进一步阐释此卦象为：“地中有山，谦。君子以裒多益寡，称物平施。”这表明君子应当效法自然的平衡之道，损有余而补不足，公平施予，以谦让为本，协调人际关系与社会资源。

从卦象结构来看，谦卦是唯一全吉的卦，六爻皆吉，象征无论在何种处境中，只要持守谦德，皆可逢凶化吉、转危为安。北宋易学家邵雍称其为“谦和忍让，尊人自卑；利用谦虚，万事可达”。得此卦者，若能守谦虚之道，前途将一片光明，而骄傲自满者则易招致失败。

在运势上，谦卦象征吉利平安、步步高升，宜守柔、宜退让、宜积蓄力量，不宜冒进张扬。在事业方面，虽然暂时可能未被重视，但凭借高尚的德行与踏实的努力，终将获得他人的认可与助力，成就一番事业。在经商方面，则预示平稳发展、逐步积累，需谨慎经营、诚心合作，方可长久获利。在求名方面，虽可能一时不被赏识，但若坚持努力与谦逊，终将脱颖而出。在婚恋方面，则提示双方需共同经营、相互谦让，方能收获美满的家庭生活。

总体而言，谦卦所传达的核心哲学是：谦逊是通达之道，是成功之基，是君子修身齐家治国平天下的重要德行。它不仅是一种处世智慧，更是一种内在修养的体现，具有极高的道德价值与实践指导意义。


In [44]:
import csv
import datetime
import os

def main():
    # 确保 data 目录存在
    if not os.path.exists('data'):
        os.makedirs('data')

    # 解析 data/raw_data.txt 得到 raw_content_data 列表
    raw_content_data = []
    with open('data/raw_data.txt', 'r', encoding='utf-8') as file:
        content = file.read()
        data_samples = content.split('\n\n')
        for sample in data_samples:
            cleaned_sample = sample.strip()
            if cleaned_sample:
                raw_content_data.append(cleaned_sample)

    # 创建带有时间戳的CSV文件名
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/zhouyi_dataset_{timestamp}.csv"

    # 创建CSV文件并写入标题行
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['content', 'summary'])

        # 循环遍历 raw_content_data 数据样例
        for raw_content in raw_content_data:
            # 调用 gen_data 方法得到 ai_message_content
            ai_message_content = gen_data(raw_content)

            # 解析 ai_message_content 得到 content 和 summary
            content, summary = dataset_parser(ai_message_content)
            
            print("Content:", content)
            print("Summary:", summary)

            # 调用 generate_question_summary_pairs 得到20组 pairs
            pairs = generate_question_summary_pairs(content, summary)

            # 将 pairs 写入 csv 文件
            for pair in pairs:
                writer.writerow(pair)


In [45]:
# 执行主函数
main()

Content: {
  "content": "谦卦",
  "summary": "在《周易》六十四卦中，谦卦象征谦逊、内敛与德行之美。卦象为异卦相叠，下卦为艮（山），上卦为坤（地），即‘地中有山’。山本应高耸于地面，而今藏于地下，寓意内在高大而不张扬，体现出谦逊不露的德性。因此，谦卦是吉利之象，象征亨通顺利，君子持此德，终将有所成就。

《象辞》指出：君子应效法谦卦之象，以谦逊自处，裁多补少，量物公平施予，体现仁义与中和之道。北宋易学家邵雍认为，谦卦体现的是谦和忍让、尊人自卑的处世智慧，只要以谦为本，诸事皆可通达。

从运势来看，得此卦者吉利平安，前程远大，尤其适合埋头苦干、不事张扬之人。在事业上虽暂未被重用，但因德行高尚，终会被人赏识；在经商方面则宜稳中求进，诚心合作，逐步积累；求名者虽初时不显，但凭借谦逊好学的精神，终能厚积薄发。

此卦亦警示世人：骄横者必遭失败，唯有谦虚者才能受益无穷。无论在事业、求名、婚恋、健康等方面，都强调谦逊、忍让与内修的重要性。因此，谦卦不仅是《周易》中唯一全吉的卦象，更是一种深具教化意义的道德哲学象征。"
}
``
Summary: {
  "content": "谦卦",
  "summary": "在《周易》六十四卦中，谦卦象征谦逊、内敛与德行之美。卦象为异卦相叠，下卦为艮（山），上卦为坤（地），即‘地中有山’。山本应高耸于地面，而今藏于地下，寓意内在高大而不张扬，体现出谦逊不露的德性。因此，谦卦是吉利之象，象征亨通顺利，君子持此德，终将有所成就。

《象辞》指出：君子应效法谦卦之象，以谦逊自处，裁多补少，量物公平施予，体现仁义与中和之道。北宋易学家邵雍认为，谦卦体现的是谦和忍让、尊人自卑的处世智慧，只要以谦为本，诸事皆可通达。

从运势来看，得此卦者吉利平安，前程远大，尤其适合埋头苦干、不事张扬之人。在事业上虽暂未被重用，但因德行高尚，终会被人赏识；在经商方面则宜稳中求进，诚心合作，逐步积累；求名者虽初时不显，但凭借谦逊好学的精神，终能厚积薄发。

此卦亦警示世人：骄横者必遭失败，唯有谦虚者才能受益无穷。无论在事业、求名、婚恋、健康等方面，都强调谦逊、忍让与内修的重要性。因此，谦卦不仅是《周易》中唯一全吉的卦象，更是一种深具教化意义的道德哲学象征。
Content: 恒卦
Summary: 恒卦在《周易》中是一个象征恒久

### 异常分析


训练第一个 epoch 时，Training Loss 比较奇怪：

```
Step	Training Loss
1	3.594100
2	4.049100
3	3.091200
4	3.381700
5	3.547800
6	2.610200
7	2.657900
8	3.163900
```

通过解析 gpt-4o-mini 生成结果发现问题

In [40]:
def gen_data(raw_content):
    """
    使用LangChain GPT-4o-mini调用处理单个数据样例。

    :param raw_content: 原始数据样例。
    :return: GPT-4o-mini模型生成的内容。
    """
    # 系统消息定义背景和任务
    system_message = SystemMessage(
        content="""
        你是中国古典哲学大师，尤其擅长周易的哲学解读。

        接下来，你收到的都是关于周易卦象的解释，你需要整理润色，并生成用于大模型训练的内容和格式。

        示例输入：

        师卦，此卦是异卦相叠，下卦为坎，上卦为坤。“师”指军队。坎为水、为险；坤为地、为顺，喻寓兵于农。兵凶战危，用兵乃圣人不得已而为之，但它可以顺利无阻碍地解决矛盾，因为顺乎形势，师出有名，故能化凶为吉。占得此卦，对于军事上率师出征非常有利，必无灾祸。师卦是天马出群之卦，以寡伏众之象。
        师卦位于讼卦之后，《序卦》之中这样解释道：“讼必有众起，故受之以师。师者，众也。”争讼的人越来越多，以致形成了军队。

        期待结果：

        content:"师卦"
        summary:"在周易中，师卦是一个极具深意的卦象，它由两个异卦相叠组成：下卦坎（水）和上卦坤（地）。这一卦象代表“师”，即军队，寓意着兵力和农力的结合。在这里，坎卦象征着水和险难，而坤卦象征着地和顺从，暗示着通过将军事力量安置于民间，可以在必要时顺利调动。

        师卦的核心哲学是：虽然兵力代表着危险和战争，但其使用应当是圣人不得已而为之的最后手段。在正确的情况下，军事力量可以顺应形势，将危险转化为吉祥。因此，在军事策略上，此卦象征着出征将会顺利，无灾祸。

        师卦紧随讼卦（争讼卦），在《序卦》中解释为“讼必有众起，故受之以师”。这意味着争端激化至众多人群的参与，形成了类似军队的集体力量。"

        返回格式要求：
        content:"{卦名}"
        summary:"{内容}"
        """
    )

    # 人类消息包含原始数据样例
    human_message = HumanMessage(
        content=raw_content
    )

    # 构建消息列表并进行模型调用
    messages = [system_message, human_message]
    ai_message = chat(messages)

    return ai_message.content

In [41]:
# 执行主函数
main()

Content: 蒙卦
Summary: 在周易中，蒙卦是一个富有启示意义的卦象，由上卦艮（山）和下卦坎（水）组成，象征着在山下涌出的泉水。此卦传达了‘蒙’即蒙昧的状态，强调智慧尚未开启、犹豫不决和缺乏果断的状态。因此，该卦常常暗示迷惑、疑虑与反复不定，初时可能面临困惑，但最终通过获得启发和教育能够通达亨利。

卦辞中提到：“不是我有求于幼稚愚昧的人，而是幼稚愚昧的人有求于我。”这说明，在更加成熟的状态中，君子应以果敢坚毅的行动去培养自身的德行。此卦象启示我们，光明的未来需要耐心、聆听他人的意见以及勇敢的行动来实现。

从不同角度分析，蒙卦在时运、事业、经商、求名、婚恋等方面均有其独特的指导意义。时运上，强调蓄积德行；事业上，尽管开始阶段可能混乱，但应通过教育与实践来转变；经商时，则需小心谨慎，注重商业道德；求名中，强调基础教育的重要性；婚恋则提醒关注对方的品德而非物质条件。蒙卦提醒我们，待机而动，审时度势，便能化解困境，迈向顺利之途。
Content: 屯卦
Summary: 屯卦象征着初生的困难与险境，它的构成由上下两个异卦组成：上卦为坎（云），下卦为震（雷）。这一卦在周易中被解释为大吉大利，意味着虽然初期艰难，但雷雨交加的环境中蕴藏着生机与希望。换言之，屯卦表达了万物萌发之初所必经历的挑战。

《象辞》强调，君子应借鉴云雷的特性，以云的恩泽和雷的威严来治理国家。这种治理需要谨慎与果敢，因屯卦本身指出，在危机和困难中宜守不宜进，需要施以耐心和努力，才能逐步排除困难，迎接成功。

从天机的角度来看，屯卦提醒我们身处困境时，务必小心翼翼，勇往直前，积极应对困难，寻求他人帮助，也必须在潜心积累后再求发展。对于事业和财运，初期创业可能会面临挫折，因此决策上应保持坚定信念，通过努力克服艰难。在婚恋方面，尽管可能经历波折，但始终坚持真诚的大步追求，能够实现美满的期望。

总之，屯卦教导我们在面临挑战时要有坚韧不拔的毅力和乐观的态度，才能在逆境中找到突破和成就的机会。
Content: 需卦
Summary: 需卦代表着等待与期待，其卦象由上卦坎（水）和下卦乾（天）组合而成，象征着云聚天上，预示着适时降雨。此卦的核心思想在于虽然时机尚未成熟，但若能耐心等待，最终会迎来好运。

需卦的本意在于应对险阻，其名虽然意含抓获俘虏，同时也体现了对时机的把握。古人有云：“云上于天，需；君